# Projeto 2 - Ciência dos Dados

Nome: Luiz Felipe Domingues Valente

Nome: João Guilherme Cintra de Freitas Almeida

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
import re 

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [16]:
#Dados de autenticação do twitter:

# Coloque aqui o identificador da conta no twitter: @jofenina

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [5]:
#Produto escolhido:
produto = 'snickers'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     msgs.append(msg.full_text.lower())
#     i += 1
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Limpando

In [8]:
tweets_tr = pd.read_excel("snickers.xlsx")
tweets_tr.head()

,Treinamento,Relevância
0,@tiagoatoc nunca pensei que me mandasses comer...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,@yenexiel ninguém compra snickers pra mim quan...,0


In [32]:
treinamento = tweets_tr.loc[:, "Treinamento"]
rel=tweets_tr.loc[:, "Relevância"]
treinamento.head()

0    @tiagoatoc nunca pensei que me mandasses comer...
1    credo q eu acordei sem paciência, e já to come...
2    samyra me deu batata frita com cheddar e bacon...
3    crlh acabei de descobrir q tem sorvete de snic...
4    @yenexiel ninguém compra snickers pra mim quan...
Name: Treinamento, dtype: object

In [10]:
def cleanup(text):
    
#  Limpeza de caracteres

    punctuation = '[!\-.:?;@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [28]:
t_limpos1 = []
t_limpos2 = []
t_limpos = []
for linha in treinamento:
    b=linha.lower()
    t_limpos1.append(cleanup(b))
for e in t_limpos1:
    a = e.strip()
    t_limpos2.append(a)
for s in t_limpos2:
    z = ' '.join(s.split())
    t_limpos.append(z)
t_limpos[0:5]

['tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente',
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã',
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas',
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem',
 'yenexiel ninguém compra snickers pra mim quando tô triste']

# Tabela com limpeza e relevância

In [41]:
t_rel_n_rel=pd.DataFrame({"Treinamento": t_limpos,"Relevância": rel})
t_rel_n_rel.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,yenexiel ninguém compra snickers pra mim quand...,0


In [44]:
t_rel= t_rel_n_rel[(t_rel_n_rel.Relevância==1)]
t_rel.head()

,Treinamento,Relevância
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
6,"rt thaii_xx sério, o anúncio da snickers no yt...",1
7,rt amandinhavr eu esperando a propaganda insup...,1
8,"não aguento mais esses anúncios da snickers, n...",1


In [45]:
t_n_rel= t_rel_n_rel[(t_rel_n_rel.Relevância==0)]
t_n_rel.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
4,yenexiel ninguém compra snickers pra mim quand...,0
5,se tu roubar uma birosca dentro da favela os c...,0
15,a casa do jovem surtado se resume a pacotes de...,0


### Tabelas de frequência

In [22]:
serie_tweets = ' '.join(t_limpos)
a = pd.Series(serie_tweets.split())
a.head()

0    tiagoatoc
1        nunca
2       pensei
3          que
4           me
dtype: object

In [23]:
a.value_counts().head()

snickers    267
de          143
e           119
um          107
eu           99
dtype: int64

In [26]:
tabela_tweets_relativa = a.value_counts(True)
tabela_tweets_relativa.head()

snickers    0.060040
de          0.032157
e           0.026760
um          0.024061
eu          0.022262
dtype: float64

In [29]:
imprime = 4
for palavra in tabela_tweets_relativa.index:
    if imprime:
        print(tabela_tweets_relativa[palavra])
        imprime-=1

0.06004047672588262
0.032156510006746124
0.026759613222397123
0.024061164830222623


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**